## **Pegasus Encoder Decoder**

Imports

In [1]:
# %pip install accelerate -U
# !pip install transformers[torch]
# !pip install accelerate -U
from src.training_utils import *

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


In [2]:
import transformers
print(transformers.__version__)

4.30.2


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the Pubmed dataset which was downloaded to the local directory 

In [3]:
from datasets import load_dataset
pubmed_dataset = load_dataset("ccdv/pubmed-summarization")#, data_dir ="/home/peppe/Desktop/Università/Projects/SAESum-Simultaneous_Abstractive_and_Extractive_Summarization/src/data/PubMed_abstractive")
print(pubmed_dataset)

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
100%|██████████| 3/3 [00:00<00:00, 957.82it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


In [4]:
print(pubmed_dataset["train"][1])
print(pubmed_dataset["test"][1])

{'article': 'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . \n preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer . \n thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management . 

Visualize some examples

Load tokenizer and model

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_checkpoint = "google/pegasus-x-base" # Use pegasus-x-base-finetuned-xsum
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer)
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print(model)

PegasusTokenizerFast(name_or_path='google/pegasus-x-base', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<

Load the metric

In [16]:
from evaluate import load
metric = load("rouge")
print(metric)

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

Create pre-processing function

In [17]:
max_input_length = 512
max_target_length = 256
#DA MODIFICARFE PER PEGASUS FORSE
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["abstract"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
preprocess_function(pubmed_dataset['train'][:1])

{'input_ids': [[114, 909, 11624, 1382, 2375, 120, 115, 2651, 110, 108, 47422, 143, 110, 38117, 233, 57468, 110, 158, 604, 404, 3629, 197, 371, 231, 195, 26364, 110, 108, 21472, 132, 11526, 70443, 111, 56744, 143, 13076, 233, 57424, 110, 158, 604, 195, 26364, 110, 108, 21472, 132, 11526, 72225, 115, 109, 1690, 1105, 110, 107, 115, 110, 35057, 114, 692, 790, 110, 58373, 281, 399, 2220, 115, 110, 116, 43449, 111, 35739, 33824, 19244, 2375, 19107, 113, 305, 76040, 110, 108, 110, 74760, 111, 48254, 110, 108, 118, 72225, 110, 108, 15713, 111, 10703, 110, 108, 4802, 110, 107, 109, 19107, 113, 40999, 790, 8225, 399, 4843, 404, 115, 40727, 8032, 6005, 135, 20177, 112, 30316, 110, 107, 53018, 21026, 692, 113, 8225, 399, 392, 115, 32043, 28064, 3264, 120, 30316, 113, 183, 4512, 135, 40999, 111, 580, 513, 1050, 110, 107, 6406, 246, 133, 3251, 233, 4087, 79876, 644, 111, 256, 319, 371, 233, 377, 3957, 113, 3085, 943, 242, 110, 107, 9939, 110, 108, 399, 5173, 962, 127, 850, 130, 109, 1146, 962, 110,

## **TRAINING**

In [18]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [19]:
tokenized_datasets = pubmed_dataset.map(preprocess_function, batched=True)
print(tokenized_datasets["train"][0]["input_ids"])

Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-83fe1a3796a65436.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-251fe14d67349aa9.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-33ed9b12aba01fbd.arrow


[114, 909, 11624, 1382, 2375, 120, 115, 2651, 110, 108, 47422, 143, 110, 38117, 233, 57468, 110, 158, 604, 404, 3629, 197, 371, 231, 195, 26364, 110, 108, 21472, 132, 11526, 70443, 111, 56744, 143, 13076, 233, 57424, 110, 158, 604, 195, 26364, 110, 108, 21472, 132, 11526, 72225, 115, 109, 1690, 1105, 110, 107, 115, 110, 35057, 114, 692, 790, 110, 58373, 281, 399, 2220, 115, 110, 116, 43449, 111, 35739, 33824, 19244, 2375, 19107, 113, 305, 76040, 110, 108, 110, 74760, 111, 48254, 110, 108, 118, 72225, 110, 108, 15713, 111, 10703, 110, 108, 4802, 110, 107, 109, 19107, 113, 40999, 790, 8225, 399, 4843, 404, 115, 40727, 8032, 6005, 135, 20177, 112, 30316, 110, 107, 53018, 21026, 692, 113, 8225, 399, 392, 115, 32043, 28064, 3264, 120, 30316, 113, 183, 4512, 135, 40999, 111, 580, 513, 1050, 110, 107, 6406, 246, 133, 3251, 233, 4087, 79876, 644, 111, 256, 319, 371, 233, 377, 3957, 113, 3085, 943, 242, 110, 107, 9939, 110, 108, 399, 5173, 962, 127, 850, 130, 109, 1146, 962, 110, 108, 278, 233,

In [20]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    wandb.log({'rouge1': result['rouge1'], 'rouge2': result['rouge2'], 'rougeL': result['rougeL'], 'rougeLsum': result['rougeLsum']})
    

    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
for param in trainer.model.model.encoder.parameters():
    param.requires_grad = False

In [23]:
trainer.train()
wandb.finish()

  8%|▊         | 1153/14991 [04:36<55:20,  4.17it/s]
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  3%|▎         | 500/14991 [01:32<44:52,  5.38it/s]

{'loss': 2.8529, 'learning_rate': 0.0004834567407110934, 'epoch': 0.03}


In [ ]:
#import pegasus
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")

model_name = "google/pegasus-x-base"
model_s = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )
model_s.to(device)


ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model_s.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
"California's largest electricity provider has turned off power to hundreds of thousands of customers."


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-x-base")

In [ ]:
input_text = "Studies have shown that owning a dog has numerous benefits."

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

output_tokens = model.generate(
    input_tokens.input_ids,
    decoder_start_token_id=model.config.pad_token_id,
    max_length=50,  # Set the desired maximum length of the generated output
    num_beams=1,  # Number of beams for beam search
    early_stopping=True,  # Stop generation when all beams have reached the end token
)

# Decode the generated output tokens
decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(decoded_output)

In [ ]:

# Example input text
input_text = ["Studies have shown that owning a dog has numerous benefits.",
              "hi how are you?"]

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt",
                         truncation=True, max_length=100,
                         padding='max_length'
                         )
print(input_tokens['input_ids'].shape)
# Pass the input through the encoder
encoder_outputs = model.model.encoder(**input_tokens)

# # Access the encoder outputs
# encoder_last_hidden_state = encoder_outputs.last_hidden_state
# print(encoder_last_hidden_state.shape)